# 앙상블


앙상블 기법은 다양한 모델을 결합하여 예측 성능을 향상시키기 위한 방법이다. 대표적인 방법으로는 **보팅(Voting)**, **배깅(Bagging)**, **부스팅(Boosting)**, **스태킹(Stacking)** 이 있다.


---


**앙상블 기법 요약**


| 구분 | 개념 | 방식 | 대표 알고리즘 |
|------|------|------|----------------|
| **보팅** (Voting) | 여러 모델의 예측을 결합하여 최종 예측을 수행한다. | 다수결(하드 보팅) 또는 예측 확률 평균(소프트 보팅)으로 결정한다. | 하드 보팅, 소프트 보팅 |
| **배깅** (Bagging) | 데이터에서 여러 부트스트랩 샘플을 생성하여 모델을 학습시키고, 예측을 결합한다. | 각 모델은 독립적으로 학습되며, 결과를 평균 또는 투표로 집계한다. | 랜덤 포레스트 |
| **부스팅** (Boosting) | **이전 모델이 틀린 데이터를 중심으로 다음 모델이 학습되도록 한다.** | 순차적으로 모델을 학습시켜 성능을 점진적으로 개선한다. | GBM, XGBoost, LightGBM, CatBoost |
| **스태킹** (Stacking) | 여러 기본 모델의 예측 결과를 새로운 데이터로 활용하여 메타 모델을 학습한다. | 다양한 모델의 예측 결과를 입력으로 받아 최종 예측을 수행한다. | 로지스틱 회귀, 랜덤 포레스트, GBM 등과 메타 모델 조합 |


---


1. 보팅 (Voting)
  - 여러 모델이 예측한 결과를 결합하여 최종 결과를 도출한다.
  - 하드 보팅은 다수결 투표로, 소프트 보팅은 예측 확률의 평균을 통해 결과를 결정한다.
2. 배깅 (Bagging)
  - 부트스트랩 샘플을 통해 여러 모델을 학습시키고, 예측을 집계한다.
  - 대표적으로 랜덤 포레스트가 있으며, 과적합을 줄이고 안정적인 예측을 가능하게 한다.
3. 부스팅 (Boosting)
  - 모델을 순차적으로 학습시키며, 이전 모델이 잘못 예측한 샘플에 가중치를 부여한다.
  - 성능이 점진적으로 향상되며, 강력한 예측력을 갖는다.
4. 스태킹 (Stacking)
  - 여러 모델의 예측 결과를 기반으로 새로운 데이터를 특성으로 생성하고, 메타 모델이 이를 학습한다.
  - 다양한 모델의 강점을 조합하여 성능을 높일 수 있다.

# Voting 계열

### VotingClassifier 분류
- hard voting: 다수결
- soft voting: 평균처리

In [18]:
from statistics import LinearRegression

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

In [4]:
from sklearn.datasets import load_breast_cancer # 유방암 데이터
from sklearn.model_selection import train_test_split
# 데이터 로드
cancer = load_breast_cancer()


# 데이터 분리
X = cancer.data
y = cancer.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

print(X_train_scaled.shape, y_train.shape)
print(X_test_scaled.shape, y_test.shape)

(455, 30) (455,)
(114, 30) (114,)


## hard voting

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

# 개별 분류기 모델
lr_clf = LogisticRegression(random_state=42)
knn_clf = KNeighborsClassifier(n_neighbors=5)
dt_clf = DecisionTreeClassifier(random_state=42)


# 하드보팅 분류기
hard_voting_clf = VotingClassifier(
    estimators=[
        ('lr', lr_clf),
        ('knn', knn_clf),
        ('dt', dt_clf),
    ],
    voting='hard'
)

hard_voting_clf.fit(X_train_scaled, y_train)

print('학습셋:', hard_voting_clf.score(X_train_scaled, y_train))
print('평가셋:', hard_voting_clf.score(X_test_scaled, y_test))

학습셋: 0.9912087912087912
평가셋: 0.9824561403508771


In [17]:
# 개별 분류기 비교


# 개별 분류기 학습
lr_clf.fit(X_train_scaled, y_train)
knn_clf.fit(X_train_scaled, y_train)
dt_clf.fit(X_train_scaled, y_train)

# 예측값
lr_pred = lr_clf.predict(X_test_scaled)
knn_pred = knn_clf.predict(X_test_scaled)
dt_pred = dt_clf.predict(X_test_scaled)
hard_voting_pred = hard_voting_clf.predict(X_test_scaled)



# 평가
# 각각 모델 가지고 와서 스코어 판단
print('lr acc: ', lr_clf.score(X_test_scaled, y_test))
print('knn acc: ', knn_clf.score(X_test_scaled, y_test))
print('dt acc: ', dt_clf.score(X_test_scaled, y_test))
print('hard voting acc: ', hard_voting_clf.score(X_test_scaled, y_test))

# 정답 비교
start = 30
stop = 45


pred_df = pd.DataFrame([
    lr_pred[start:stop],
    knn_pred[start:stop],
    dt_pred[start:stop],
    hard_voting_pred[start:stop]
], index=['lr', 'knn', 'dt', 'hard_voting'])  # 표로 보이긴 하는데 0,1,2,3번이 뭔지 판단하기 위해서


pred_df

lr acc:  0.9824561403508771
knn acc:  0.9736842105263158
dt acc:  0.9210526315789473
hard voting acc:  0.9824561403508771


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
lr,1,1,1,1,0,0,1,1,1,0,1,1,1,0,0
knn,1,1,1,1,0,1,1,1,1,0,1,1,1,0,0
dt,1,1,1,0,0,0,1,0,1,0,1,1,1,0,0
hard_voting,1,1,1,1,0,0,1,1,1,0,1,1,1,0,0


## soft voting

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier


lr_clf = LogisticRegression(random_state=42)
# 개별 분류기 모델
knn_clf = KNeighborsClassifier(n_neighbors=5)
dt_clf = DecisionTreeClassifier(random_state=42)


# 소프트보팅 분류기
soft_voting_clf = VotingClassifier(
    estimators=[
        ('lr', lr_clf),
        ('knn', knn_clf),
        ('dt', dt_clf),
    ],
    voting='soft' # 하드보팅은 다수결이었다면 얘는 평균을 냄
)

soft_voting_clf.fit(X_train_scaled, y_train)

print('학습셋:', soft_voting_clf.score(X_train_scaled, y_train))
print('평가셋:', soft_voting_clf.score(X_test_scaled, y_test))

학습셋: 0.9956043956043956
평가셋: 0.9649122807017544


In [24]:
# 개별 분류기 비교


# 개별 분류기 학습
lr_clf.fit(X_train_scaled, y_train)
knn_clf.fit(X_train_scaled, y_train)
dt_clf.fit(X_train_scaled, y_train)

# 확률값
lr_pred_proba = lr_clf.predict_proba(X_test_scaled)
knn_pred_proba = knn_clf.predict_proba(X_test_scaled)
dt_pred_proba = dt_clf.predict_proba(X_test_scaled)
soft_voting_pred = soft_voting_clf.predict_proba(X_test_scaled)



# 평가
# 각각 모델 가지고 와서 스코어 판단
print('lr acc: ', lr_clf.score(X_test_scaled, y_test))
print('knn acc: ', knn_clf.score(X_test_scaled, y_test))
print('dt acc: ', dt_clf.score(X_test_scaled, y_test))
print('soft voting acc: ', soft_voting_clf.score(X_test_scaled, y_test))

# 정답 비교
start = 30
stop = 40


pred_df = pd.DataFrame([
    lr_pred_proba[start:stop, 1],
    knn_pred_proba[start:stop, 1],
    dt_pred_proba[start:stop, 1],
    soft_voting_pred[start:stop, 1]
], index=['lr', 'knn', 'dt', 'soft_voting'])  # 표로 보이긴 하는데 0,1,2,3번이 뭔지 판단하기 위해서


pred_df.loc['avg'] =(pred_df.loc['lr'] + pred_df.loc['knn'] + pred_df.loc['dt']) / 3
                                                                        # ==평균이됨
pred_df.loc['soft_voting_pred'] = soft_voting_clf.predict(X_test_scaled[start:stop])

pred_df

lr acc:  0.9824561403508771
knn acc:  0.9736842105263158
dt acc:  0.9210526315789473
soft voting acc:  0.9649122807017544


,0,1,2,3,4,5,6,7,8,9
lr,0.995182,0.999997,0.987569,0.994157,0.000016,0.169705,0.998431,0.992515,0.593540,0.000031
knn,1.000000,1.000000,1.000000,1.000000,0.000000,0.600000,1.000000,1.000000,0.800000,0.000000
dt,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
soft_voting,0.998394,0.999999,0.995856,0.664719,0.000005,0.256568,0.999477,0.664172,0.797847,0.000010
avg,0.998394,0.999999,0.995856,0.664719,0.000005,0.256568,0.999477,0.664172,0.797847,0.000010
soft_voting_pred,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000


## VotingRegressor
- 회귀 예측에는 soft 평균방식만 지원

In [27]:
from sklearn.datasets import fetch_california_housing

X, y = fetch_california_housing(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import VotingRegressor

# 개별 회귀예측기

lin_reg = LinearRegression()
knn_reg = KNeighborsRegressor(n_neighbors=5)
dt_reg = DecisionTreeRegressor(random_state=42)

voting_reg = VotingRegressor(
    estimators=[
        ('lin', lin_reg),
        ('knn', knn_reg),
        ('dt', dt_reg)
    ]
)

voting_reg.fit(X_train_scaled, y_train)

# -> 실제 모델에다가 학습 시킨게 아니고 복사본에다가 시킴


,estimators,"[('lin', ...), ('knn', ...), ...]"
,weights,None
,n_jobs,None
,verbose,False
,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False
,n_neighbors,5
,weights,'uniform'


In [35]:
from sklearn.metrics import root_mean_squared_error, r2_score

# 원본 모델 학습
lin_reg.fit(X_train_scaled, y_train)
knn_reg.fit(X_train_scaled, y_train)
dt_reg.fit(X_train_scaled, y_train)

for reg in [lin_reg, knn_reg, dt_reg, voting_reg]:
    class_name = reg.__class__.__name__
    y_pred = reg.predict(X_test_scaled)

    print(f'{class_name} : '
          f'RMSE= {root_mean_squared_error(y_test, y_pred)},'
          f'R2={r2_score(y_test, y_pred)}')

LinearRegression : RMSE= 0.7455813830127763,R2=0.575787706032451
KNeighborsRegressor : RMSE= 0.6575877238850522,R2=0.6700101862970989
DecisionTreeRegressor : RMSE= 0.7028289572288925,R2=0.6230424613065773
VotingRegressor : RMSE= 0.591653369272247,R2=0.7328668261712341
